# `concurrent.futures` — Launching parallel tasks

https://docs.python.org/3/library/concurrent.futures.html

In [2]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [4]:
help(ThreadPoolExecutor)

Help on class ThreadPoolExecutor in module concurrent.futures.thread:

class ThreadPoolExecutor(concurrent.futures._base.Executor)
 |  ThreadPoolExecutor(max_workers=None, thread_name_prefix='', initializer=None, initargs=())
 |  
 |  This is an abstract base class for concrete asynchronous executors.
 |  
 |  Method resolution order:
 |      ThreadPoolExecutor
 |      concurrent.futures._base.Executor
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, max_workers=None, thread_name_prefix='', initializer=None, initargs=())
 |      Initializes a new ThreadPoolExecutor instance.
 |      
 |      Args:
 |          max_workers: The maximum number of threads that can be used to
 |              execute the given calls.
 |          thread_name_prefix: An optional name prefix to give our threads.
 |          initializer: A callable used to initialize worker threads.
 |          initargs: A tuple of arguments to pass to the initializer.
 |  
 |  shutdown(self, wait=T

In [3]:
help(ProcessPoolExecutor)

Help on class ProcessPoolExecutor in module concurrent.futures.process:

class ProcessPoolExecutor(concurrent.futures._base.Executor)
 |  ProcessPoolExecutor(max_workers=None, mp_context=None, initializer=None, initargs=())
 |  
 |  This is an abstract base class for concrete asynchronous executors.
 |  
 |  Method resolution order:
 |      ProcessPoolExecutor
 |      concurrent.futures._base.Executor
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, max_workers=None, mp_context=None, initializer=None, initargs=())
 |      Initializes a new ProcessPoolExecutor instance.
 |      
 |      Args:
 |          max_workers: The maximum number of processes that can be used to
 |              execute the given calls. If None or not given then as many
 |              worker processes will be created as the machine has processors.
 |          mp_context: A multiprocessing context to launch the workers. This
 |              object should provide SimpleQueue, Queue and 

In [5]:
import threading
import os
from functools import wraps
import time

def thread_info(fn):

    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"{threading.current_thread().name}. Start function: {fn.__name__}. {args = }, {kwargs = }")
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"{threading.current_thread().name}. End function: {fn.__name__}")

    return wrapper


def process_info(fn):

    @wraps(fn)
    def wrapper(*args, **kwargs):
        print(f"Process ID: {os.getpid()}. Parrent Process: {os.getppid()}. Start function: {fn.__name__}. {args = }, {kwargs = }")
        try:
            return fn(*args, **kwargs)
        finally:
            print(f"Process ID: {os.getpid()}. End function: {fn.__name__}")

    return wrapper

## `submit(fn, /, *args, **kwargs`)

In [9]:
from concurrent.futures import ThreadPoolExecutor

@process_info
@thread_info
def squares(x):
    return x ** 2


with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.submit(squares, 1)
    
    print(f"{future = }, {future.result() = }")


Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (1,), kwargs = {}
ThreadPoolExecutor-2_0. Start function: squares. args = (1,), kwargs = {}
ThreadPoolExecutor-2_0. End function: squares
Process ID: 12742. End function: squares
future = <Future at 0x7f598c690100 state=finished returned int>, future.result() = 1


In [10]:
from concurrent.futures import ProcessPoolExecutor


@process_info
@thread_info
def squares(x):
    return x ** 2


with ProcessPoolExecutor(max_workers=1) as executor:
    future = executor.submit(squares, 1)
    
    print(f"{future = }, {future.result() = }")

Process ID: 19544. Parrent Process: 12742. Start function: squares. args = (1,), kwargs = {}
MainThread. Start function: squares. args = (1,), kwargs = {}
MainThread. End function: squares
Process ID: 19544. End function: squares
future = <Future at 0x7f598c680040 state=running>, future.result() = 1


## `map(func, *iterables, timeout=None, chunksize=1)`

In [14]:
from concurrent.futures import ThreadPoolExecutor
import time

@process_info
@thread_info
def squares(x):
    return x ** 2


with ThreadPoolExecutor(max_workers=1) as executor:
    future_result = executor.map(squares, range(10))
   
    print(f"{future_result = }")
    for item in future_result:
        print(item)
        

Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (0,), kwargs = {}future_result = <generator object Executor.map.<locals>.result_iterator at 0x7f598c2e2eb0>

ThreadPoolExecutor-6_0. Start function: squares. args = (0,), kwargs = {}
ThreadPoolExecutor-6_0. End function: squares
Process ID: 12742. End function: squares
Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (1,), kwargs = {}
ThreadPoolExecutor-6_0. Start function: squares. args = (1,), kwargs = {}
ThreadPoolExecutor-6_0. End function: squares
Process ID: 12742. End function: squares
Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (2,), kwargs = {}
ThreadPoolExecutor-6_0. Start function: squares. args = (2,), kwargs = {}
ThreadPoolExecutor-6_0. End function: squares
Process ID: 12742. End function: squares
Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (3,), kwargs = {}
ThreadPoolExecutor-6_0. Start function: squares. args = (3

In [20]:
from concurrent.futures import ThreadPoolExecutor
import time

@process_info
@thread_info
def squares(x):
    return x ** 2


with ThreadPoolExecutor(max_workers=None) as executor:
    future_result = executor.map(squares, range(20))
   
    print(f"{future_result = }")


    for item in future_result:
        print(item)
    print(f" >>>>>>> {executor._max_workers = }")

Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (0,), kwargs = {}
ThreadPoolExecutor-12_0. Start function: squares. args = (0,), kwargs = {}
ThreadPoolExecutor-12_0. End function: squares
Process ID: 12742. End function: squares
Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (1,), kwargs = {}
ThreadPoolExecutor-12_0. Start function: squares. args = (1,), kwargs = {}
ThreadPoolExecutor-12_0. End function: squares
Process ID: 12742. End function: squares
Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (2,), kwargs = {}
ThreadPoolExecutor-12_0. Start function: squares. args = (2,), kwargs = {}
ThreadPoolExecutor-12_0. End function: squares
Process ID: 12742. End function: squares
Process ID: 12742. Parrent Process: 4691. Start function: squares. args = (3,), kwargs = {}
ThreadPoolExecutor-12_0. Start function: squares. args = (3,), kwargs = {}
ThreadPoolExecutor-12_0. End function: squares
Process ID: 12742. En

In [21]:
from concurrent.futures import ProcessPoolExecutor


@process_info
@thread_info
def squares(x):
    return x ** 2


with ProcessPoolExecutor(max_workers=None) as executor:
    future = executor.map(squares, range(10))
    
    print(f"{future = }")
    
    for item in future:
        print(f"{item = }")
        
    print(f" >>>>> {executor._max_workers = }")

Process ID: 19662. Parrent Process: 12742. Start function: squares. args = (3,), kwargs = {}Process ID: 19661. Parrent Process: 12742. Start function: squares. args = (2,), kwargs = {}Process ID: 19663. Parrent Process: 12742. Start function: squares. args = (4,), kwargs = {}Process ID: 19660. Parrent Process: 12742. Start function: squares. args = (1,), kwargs = {}Process ID: 19659. Parrent Process: 12742. Start function: squares. args = (0,), kwargs = {}

Process ID: 19664. Parrent Process: 12742. Start function: squares. args = (5,), kwargs = {}

MainThread. Start function: squares. args = (1,), kwargs = {}MainThread. Start function: squares. args = (0,), kwargs = {}
MainThread. Start function: squares. args = (4,), kwargs = {}MainThread. Start function: squares. args = (5,), kwargs = {}

MainThread. Start function: squares. args = (3,), kwargs = {}MainThread. End function: squares
MainThread. End function: squares
MainThread. End function: squares

MainThread. End function: squares

In [22]:
from concurrent.futures import ThreadPoolExecutor
import operator
import itertools


with ThreadPoolExecutor(max_workers=1) as executor:
    future = executor.map(operator.add, range(10), itertools.repeat(1))
    
    print(f"{future = }")
    
    for item in future:
        print(f"{item = }")

future = <generator object Executor.map.<locals>.result_iterator at 0x7f598c07beb0>
item = 1
item = 2
item = 3
item = 4
item = 5
item = 6
item = 7
item = 8
item = 9
item = 10
